![CoSAppLogo](images/cosapp.svg) **CoSApp** examples:

# Optimization

Preliminary note:

This case is taken from [OpenMDAO](http://openmdao.org/twodocs/versions/latest/examples/betz_limit/betz.html). OpenMDAO is an open-source computing platform for system analysis and multidisciplinary optimization developed by the NASA. Its philosophy shares some of the goals of CoSApp. Thus, this example is also an opportunity to compare both libraries.

OpenMDAO is licensed under [Apache License](https://github.com/OpenMDAO/OpenMDAO/blob/master/LICENSE.txt).

## Case description

**Optimizing an Actuator Disk Model to Find Betz Limit for Wind Turbines**

The Betz limit is the theoretical maximum amount of kinetic energy that a wind turbine can extract from the flow. This limit was derived analytically by Albert Betz in 1919, but it can also be found numerically using an optimizer and a simple actuator disk model for a wind-turbine.

## Creating the elements

In [ ]:
from cosapp.base import System

class ActuatorDisc(System):
    """Simple wind turbine model based on actuator disc theory"""

    def setup(self):

        # Inputs
        self.add_inward('a', 0.5, desc="Induced Velocity Factor")
        self.add_inward('area', 10.0, unit="m**2", desc="Rotor disc area")
        self.add_inward('rho', 1.225, unit="kg/m**3", desc="air density")
        self.add_inward('Vu', 10.0, unit="m/s",
            desc="Freestream air velocity, upstream of rotor")

        # Outputs
        self.add_outward('Vr', 0.0, unit="m/s",
            desc="Air velocity at rotor exit plane")
        self.add_outward('Vd', 0.0, unit="m/s",
            desc="Slipstream air velocity, downstream of rotor")
        self.add_outward('Ct', 0.0, desc="Thrust Coefficient")
        self.add_outward('thrust', 0.0, unit="N",
            desc="Thrust produced by the rotor")
        self.add_outward('Cp', 0.0, desc="Power Coefficient")
        self.add_outward('power', 0.0, unit="W",
            desc="Power produced by the rotor")

    def compute(self):
        """
        Considering the entire rotor as a single disc that extracts
        velocity uniformly from the incoming flow and converts it to
        power.
        """
        a = self.a
        Vu = self.Vu

        qA = 0.5 * self.rho * self.area * Vu**2

        self.Vd = Vd = Vu * (1 - 2 * a)
        self.Vr = 0.5 * (Vu + Vd)

        self.Ct = Ct = 4 * a * (1 - a)
        self.thrust = Ct * qA

        self.Cp = Cp = Ct * (1 - a)
        self.power = Cp * qA * Vu


## Setting the problem

In [ ]:
# build the model
prob = ActuatorDisc('a_disk')

prob.a = .5
prob.area = 10.0
prob.rho = 1.225
prob.Vu = 10.0

## Solving the optimization

In [ ]:
from cosapp.drivers import Optimizer

# setup the optimization
opt = prob.add_driver(Optimizer('optimization', method='SLSQP'))

opt.runner.add_unknown('a', lower_bound=0., upper_bound=1.)
opt.runner.add_unknown('area', lower_bound=0., upper_bound=1.)
# negative one so we maximize the objective
opt.runner.set_objective('-Cp')

prob.run_drivers()

## Validation 

Validation of minimum value vs. OpenMDAO results

In [ ]:
assert abs(prob['Cp'] / 0.59259259 - 1) < 1e-6
assert abs(prob['a'] / 0.33335528 - 1) < 1e-6
assert abs(prob['area'] - 1) < 1e-6